In [247]:
import numpy as np

In [248]:
def mu_X(X, M):
    """Drift coefficient for X process"""
    return 0.1 * (np.sqrt(np.minimum(M, np.exp(X))) - 1) - 1/8

def sigma_X(X):
    """Diffusion coefficient for X process (constant)"""
    return 0.5

In [249]:
#### Global Params
dimension = 1
sigma =  0.5 * np.identity(1) ### Needed to compute Beta
mu_max = 5 ### Needed to compute Beta
mu_borne_lipsch = 0.1 ### Needed to compute Beta
T = 1.0

In [250]:
def return_gamma(sigma,mu_max,d,T):
    terme_1 = (1 + mu_max * np.sqrt(T))**2
    terme_2 = np.trace(np.linalg.inv(sigma*np.transpose(sigma)))
    terme_3 = 2*(3*d + d*(d-1))
    return terme_1 * terme_2 * terme_3

def return_beta(gamma,mu_borne_lipsch,T):
    Beta_star = np.sqrt(gamma * mu_borne_lipsch**2 * T + (1/4) * T**2) - 1/(2*T)
    return Beta_star

gamma = return_gamma(sigma,mu_max,dimension,T)
beta = return_beta(gamma,mu_borne_lipsch,T)

print(f"parameter gamma is {gamma}")
print(f"parameter beta is {beta}")

parameter gamma is 864.0
parameter beta is 2.481610303175115


In [251]:
###  To introduce our unbiased simulation algorithm, let us first introduce a random discrete time grid.
#  Let β>0 be a fixed positive constant, (τi)i>0 be a sequence of i.i.d. 

def generate_timing_and_brownian_matrices(N_mc, T, beta, n_max=None):
    """
    Fully vectorized: returns arrival times, dt, Brownian increments, masks.
    
    Generates random discrete time grids following equation (2.4): T_k := (Σ τ_i) ∧ T
    This naturally creates: T_1, T_2, ..., T_{N_T}, T, T, T, ...
    
    Args:
        N_mc: Number of Monte Carlo trajectories
        T: Final time horizon
        beta: Intensity parameter for exponential arrivals (rate parameter)
        n_max: Maximum number of jumps (default: estimate based on E[N_T])
    
    Returns:
        T_matrix: Arrival times clipped to T (N_mc × n_max)
        dt_matrix: Time increments ΔT_k (N_mc × n_max)
        dW_matrix: Brownian increments ΔW (N_mc × n_max)
        valid_mask: Boolean mask for unique steps (not repeated T)
        n_grids: Number of arrivals before T (N_mc,)
    """
    if n_max is None:
        n_max = int(3 * beta * T) + 12

    # Exponential increments: τ_i ~ Exp(β), so scale = 1/β
    tau_matrix = np.random.exponential(scale=1/beta, size=(N_mc, n_max))

    # Cumulative sum: Σ_{i=1}^k τ_i
    T_cumsum = np.cumsum(tau_matrix, axis=1)
    
    # Equation (2.4): T_k := (Σ τ_i) ∧ T (min with T)
    T_matrix = np.minimum(T_cumsum, T)
    
    # Compute dt_k = T_k - T_{k-1}
    T_matrix_shifted = np.column_stack([np.zeros(N_mc), T_matrix[:, :-1]])
    dt_matrix = T_matrix - T_matrix_shifted
    
    # Valid mask: where T_k != T_{k-1} (i.e., not a repeated T)
    valid_mask = dt_matrix > 1e-14
    
    # N_T = number of steps before reaching T
    n_grids = np.sum(valid_mask, axis=1)
    
    # Brownian increments: ΔW_{T_{k+1}} ~ N(0, ΔT_{k+1})
    dW_matrix = np.random.randn(N_mc, n_max) * np.sqrt(dt_matrix)
    
    # Zero out invalid entries (after reaching T)
    dW_matrix[~valid_mask] = 0
    dt_matrix[~valid_mask] = 0

    return T_matrix, dt_matrix, dW_matrix, valid_mask, n_grids


In [252]:

# Générer les matrices pour 10 trajectoires
T_matrix, dt_matrix, dW_matrix, valid_mask, n_grids = generate_timing_and_brownian_matrices(10000, T, beta)

# n_grids contient le nombre d'entrées valides pour chaque trajectoire
print(f"Nombre d'entrées valides par trajectoire: {n_grids}")
print(f"Nombre maximum d'entrées valides: {np.max(n_grids)}")
print(f"Nombre minimum d'entrées valides: {np.min(n_grids)}")
print(f"Nombre moyen d'entrées valides: {np.mean(n_grids):.2f}")
print(f"Valeur théorique attendue (beta * T): {beta * T:.2f}")

Nombre d'entrées valides par trajectoire: [3 3 4 ... 4 3 3]
Nombre maximum d'entrées valides: 12
Nombre minimum d'entrées valides: 1
Nombre moyen d'entrées valides: 3.46
Valeur théorique attendue (beta * T): 2.48


## Step 1: Restructured Algorithm

Building the simulation following the paper's logic:
1. Generate grid and take first step (X₀ → X₁)
2. Identify trajectories that are done (n_grids == 1, meaning N_T = 0)
3. For remaining trajectories, iterate and compute weights

## KEY INSIGHT FROM THE PAPER

Looking at equation (2.7) again:

$$\bar{w}_k := \frac{(\mu(T_k, \tilde{X}_{T_k}) - \mu(T_{k-1}, \tilde{X}_{T_{k-1}})) \cdot (\sigma_0^{\dagger})^{-1} \Delta W_{T_{k+1}}}{\Delta T_{k+1}}$$

**The weight w̄_k uses ΔW_{T_{k+1}} and ΔT_{k+1}** - the increments of step k→k+1!

So for step k (from T_k to T_{k+1}):
- We compute μ at T_k
- The weight w̄_k uses the drift difference μ_k - μ_{k-1}
- BUT uses dW and dt from THIS SAME STEP (k→k+1)

Not the next step! The subscript is T_{k+1} because that's the END of the current interval!

## FINAL CORRECT IMPLEMENTATION

Now that we understand: **w̄_k uses dW and dt from the CURRENT interval [T_k, T_{k+1}]**

In [255]:
def Run_Unbiased_MC_Final(N_mc, X_0, beta, mu_func, sigma_0, M, T):
    """
    FINAL CORRECT implementation.
    
    Key understanding:
    - Grid: T_0=0, T_1, ..., T_{N_T}, T_{N_T+1}=T
    - Euler steps: k = 0, 1, ..., N_T (total N_T+1 steps)
    - Weights: w̄_k for k = 1, ..., N_T (total N_T weights)
    - w̄_k uses (μ_k - μ_{k-1}) and (ΔW_k, ΔT_k) from interval [T_k, T_{k+1}]
    """
    # Generate grids
    T_matrix, dt_matrix, dW_matrix, valid_mask, n_grids = \
        generate_timing_and_brownian_matrices(N_mc, T, beta)
    n_max = T_matrix.shape[1]
    
    # N_T = number of random arrivals before T
    # n_grids = total steps = N_T + 1
    N_T = n_grids - 1
    
    # Initialize
    X_k = np.full(N_mc, X_0)  # Current state
    X_km1 = np.full(N_mc, X_0)  # Previous state
    X_T_NT = np.full(N_mc, X_0)  # State at T_{N_T}
    
    w_product = np.ones(N_mc)
    
    # Main loop: k = 0, 1, ..., N_T
    for k in range(n_max):
        # Active paths at step k
        active = (k < n_grids)
        if not np.any(active):
            break
        
        dt_k = dt_matrix[:, k]
        dW_k = dW_matrix[:, k]
        
        # Compute drift at current state
        mu_k = mu_func(X_k, M)
        
        # Weight computation for k >= 1 and k <= N_T
        # For step k: T_k → T_{k+1}, compute w̄_k if k is within [1, N_T]
        # This means: k >= 1 AND k < n_grids - 1 (since last step is T_{N_T} → T, which is k = N_T)
        # Actually, if k = N_T, we're going from T_{N_T} to T, and we DO compute w̄_{N_T}
        # Wait, let me think...
        
        # If N_T = 2, then n_grids = 3
        # Steps are: k=0 (T_0→T_1), k=1 (T_1→T_2), k=2 (T_2→T_3=T)
        # Weights are: w̄_1 (for k=1), w̄_2 (for k=2)
        # So weights are computed for k=1 and k=2, i.e., k >= 1 and k <= N_T
        # That's: k >= 1 and k <= n_grids - 1
        
        if k >= 1:
            # Compute weight for ALL active paths where k <= N_T
            compute_weight = active & (k <= N_T)
            
            if np.any(compute_weight):
                mu_km1 = mu_func(X_km1, M)
                drift_diff = mu_k - mu_km1
                
                # w̄_k = (μ_k - μ_{k-1}) · σ₀⁻¹ · ΔW_k / ΔT_k
                w_k = np.ones(N_mc)
                safe = dt_k > 1e-14
                w_k[compute_weight & safe] = (
                    drift_diff[compute_weight & safe]
                    * dW_k[compute_weight & safe]
                    / (sigma_0 * dt_k[compute_weight & safe])
                )
                
                w_product[compute_weight] *= w_k[compute_weight]
        
        # Save X_{T_{N_T}} (state at last random arrival before T)
        # This is BEFORE taking the step from T_{N_T} to T
        # Happens when k = N_T (about to take final step)
        is_at_NT = (k == N_T) & (N_T > 0)
        X_T_NT = np.where(is_at_NT, X_k, X_T_NT)
        
        # Euler step: X_{k+1} = X_k + μ_k·ΔT_k + σ_0·ΔW_k
        X_next = X_k + mu_k * dt_k + sigma_0 * dW_k
        
        # Update for next iteration
        X_km1 = np.where(active, X_k, X_km1)
        X_k = np.where(active, X_next, X_k)
    
    # Final state
    X_T = X_k
    
    # Final weights (Equation 2.6)
    weights = np.exp(beta * T) * (beta ** (-N_T.astype(float))) * w_product
    
    return X_T, X_T_NT, N_T, weights

In [256]:
# TEST FINAL IMPLEMENTATION
import time

np.random.seed(123)

K = 1.0
M_param = 4.0
N_mc = 500000
beta_use = beta

print("="*70)
print("FINAL CORRECT IMPLEMENTATION TEST")
print("="*70)

start = time.time()
X_T, X_T_NT, N_T, weights = Run_Unbiased_MC_Final(
    N_mc=N_mc,
    X_0=0.0,
    beta=beta_use,
    mu_func=mu_X,
    sigma_0=0.5,
    M=M_param,
    T=1.0
)
elapsed = time.time() - start

# Payoffs
g_X_T = np.maximum(np.exp(X_T) - K, 0)
g_X_T_NT = np.maximum(np.exp(X_T_NT) - K, 0)

# Equation (2.6): ψ̃ = weights * [g(X_T) - g(X_{T_{N_T}}) * 1_{N_T>0}]
indicator = (N_T > 0).astype(float)
payoff_estimator = weights * (g_X_T - g_X_T_NT * indicator)

V_unbiased = np.mean(payoff_estimator)
std_error = np.std(payoff_estimator) / np.sqrt(N_mc)

print(f"\n⏱  Simulation: {elapsed:.2f}s ({N_mc/elapsed:.0f} paths/sec)")
print(f"\n📊 Grid Statistics:")
print(f"   N_T range: [{N_T.min()}, {N_T.max()}]")
print(f"   N_T mean: {N_T.mean():.2f} (expected: {beta_use:.2f})")
print(f"   N_T=0 count: {np.sum(N_T==0)} ({100*np.sum(N_T==0)/N_mc:.1f}%)")

print(f"\n⚖️  Weights:")
print(f"   Range: [{weights.min():.4f}, {weights.max():.4f}]")
print(f"   Mean: {weights.mean():.4f}")

print(f"\n💰 Payoffs:")
print(f"   g(X_T) mean: {g_X_T.mean():.6f}")
print(f"   g(X_{{T_{{N_T}}}}) mean: {g_X_T_NT.mean():.6f}")

print(f"\n" + "="*70)
print(f"📈 OPTION PRICE: {V_unbiased:.6f} ± {std_error:.6f}")
print(f"   95% CI: [{V_unbiased - 1.96*std_error:.6f}, {V_unbiased + 1.96*std_error:.6f}]")
print("="*70)

FINAL CORRECT IMPLEMENTATION TEST

⏱  Simulation: 3.08s (162571 paths/sec)

📊 Grid Statistics:
   N_T range: [0, 13]
   N_T mean: 2.48 (expected: 2.48)
   N_T=0 count: 41649 (8.3%)

⚖️  Weights:
   Range: [-52.9187, 149.2731]
   Mean: 0.9969

💰 Payoffs:
   g(X_T) mean: 0.201285
   g(X_{T_{N_T}}) mean: 0.148846

📈 OPTION PRICE: 0.205696 ± 0.002192
   95% CI: [0.201400, 0.209992]


## Simplified Clean Implementation

Removing unnecessary complexity while keeping correctness.

In [257]:
def Run_Unbiased_MC(N_mc, X_0, beta, mu_func, sigma_0, M, T):
    """
    Unbiased Monte Carlo for SDEs with random grids.
    
    Implements equations (2.5), (2.6), (2.7) from the paper.
    """
    # Generate random grids
    T_matrix, dt_matrix, dW_matrix, _, n_grids = \
        generate_timing_and_brownian_matrices(N_mc, T, beta)
    
    N_T = n_grids - 1  # Number of random arrivals before T
    
    # Initialize
    X = np.full(N_mc, X_0)
    X_prev = np.full(N_mc, X_0)
    X_T_NT = np.full(N_mc, X_0)
    w_product = np.ones(N_mc)
    
    # Main loop
    for k in range(T_matrix.shape[1]):
        active = (k < n_grids)
        if not np.any(active):
            break
        
        mu_k = mu_func(X, M)
        
        # Compute weights for k >= 1 and k <= N_T
        if k >= 1:
            compute_w = active & (k <= N_T)
            if np.any(compute_w):
                mu_prev = mu_func(X_prev, M)
                w = (mu_k - mu_prev) * dW_matrix[:, k] / (sigma_0 * dt_matrix[:, k])
                w_product[compute_w] *= w[compute_w]
        
        # Save state at last random arrival
        save_NT = (k == N_T) & (N_T > 0)
        X_T_NT = np.where(save_NT, X, X_T_NT)
        
        # Euler step
        X_next = X + mu_k * dt_matrix[:, k] + sigma_0 * dW_matrix[:, k]
        
        X_prev[active] = X[active]
        X[active] = X_next[active]
    
    # Final weights
    weights = np.exp(beta * T) * (beta ** (-N_T.astype(float))) * w_product
    
    return X, X_T_NT, N_T, weights

In [258]:
# Quick test of simplified version
np.random.seed(123)

X_T, X_T_NT, N_T, weights = Run_Unbiased_MC(
    N_mc=500000,
    X_0=0.0,
    beta=beta,
    mu_func=mu_X,
    sigma_0=0.5,
    M=4.0,
    T=1.0
)

# Compute option price
g_X_T = np.maximum(np.exp(X_T) - 1.0, 0)
g_X_T_NT = np.maximum(np.exp(X_T_NT) - 1.0, 0)
indicator = (N_T > 0).astype(float)
payoff = weights * (g_X_T - g_X_T_NT * indicator)

print(f"Option Price: {np.mean(payoff):.6f} ± {np.std(payoff)/np.sqrt(len(payoff)):.6f}")

C:\Users\jacqu\AppData\Local\Temp\ipykernel_26368\2105227672.py:32: RuntimeWarning: invalid value encountered in divide
  w = (mu_k - mu_prev) * dW_matrix[:, k] / (sigma_0 * dt_matrix[:, k])


Option Price: 0.205696 ± 0.002192
